# Gymnasium, SB3 실습

목표:
- Gymnasium 환경 생성 및 설정
- SB3 agent 생성 및 학습
- agent 성능 시각화

## 1. 라이브러리 import

In [18]:
import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO  # SB3 알고리즘 중 하나인 PPO 사용
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor


## 2. 환경 생성

In [19]:
env_name = "CartPole-v1"
env = gym.make(env_name, render_mode="rgb_array")

## 3. 환경 시각화

In [20]:
observation, info = env.reset()

for _ in range(10):
    env.render()
    action = env.action_space.sample()  # 임의의 액션을 선택
    observation, reward, terminated, truncated, info = env.step(action)
    

    if terminated or truncated:
        observation, info = env.reset()

env.close()

## 4. Random Agent로 환경 테스트

In [21]:
# Random agent가 CartPole 환경에서 어떻게 수행하는지 확인
observation, info = env.reset()
total_reward = 0


while True:
    action = env.action_space.sample()  # 임의의 액션을 선택
    obs, reward, terminated, truncated, info = env.step(action)  # 액션을 실행하고 결과 확인
    total_reward += reward

    if terminated or truncated:
        print(f"Total Reward: {total_reward}")
        break

Total Reward: 23.0


## 5. 모델 생성 및 학습

- EvalCallback을 사용하여 학습 과정 log
- tensorboard로 학습 과정 시각화

In [22]:
env = make_vec_env("CartPole-v1", n_envs=4)

# PPO 모델 생성
model = PPO("MlpPolicy", env, verbose=1)

# 학습 시 콜백 설정
eval_env = Monitor(gym.make("CartPole-v1"))
eval_callback = EvalCallback(eval_env, best_model_save_path='./logs/',
                             log_path='./logs/', eval_freq=1000,
                             deterministic=True, render=False)

# 학습
model.learn(total_timesteps=10000, callback=eval_callback)

# 학습 완료 후 모델 저장
model.save("ppo_cartpole")

Using cpu device
Eval num_timesteps=4000, episode_reward=9.60 +/- 0.49
Episode length: 9.60 +/- 0.49
---------------------------------
| eval/              |          |
|    mean_ep_length  | 9.6      |
|    mean_reward     | 9.6      |
| time/              |          |
|    total_timesteps | 4000     |
---------------------------------
New best mean reward!
Eval num_timesteps=8000, episode_reward=9.20 +/- 0.75
Episode length: 9.20 +/- 0.75
---------------------------------
| eval/              |          |
|    mean_ep_length  | 9.2      |
|    mean_reward     | 9.2      |
| time/              |          |
|    total_timesteps | 8000     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 21.1     |
|    ep_rew_mean     | 21.1     |
| time/              |          |
|    fps             | 3150     |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 8192     |
-----------

## 6. 학습된 모델 시각화

In [25]:
import matplotlib.pyplot as plt
import os
# 학습된 모델 불러오기
model = PPO.load("ppo_cartpole")

# 학습된 모델 평가
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, render=True)
print(f"평균 보상: {mean_reward}, 보상의 표준편차: {std_reward}")

# 평가 시각화
env = gym.make("CartPole-v1", render_mode='rgb_array')
observation, info = env.reset()
frames = []

for _ in range(1000):
    action, _states = model.predict(observation, deterministic=True)
    observation, rewards, terminated, truncated, info = env.step(action)
    
    frame = env.render()
    frames.append(frame)

    if terminated or truncated:
        observation, info = env.reset()

env.close()

import moviepy.editor as mpy

video_dir = './videos'
os.makedirs(video_dir, exist_ok=True)
clip = mpy.ImageSequenceClip(frames, fps=30)
clip.write_videofile(f"{video_dir}/cartpole.mp4")

평균 보상: 348.5, 보상의 표준편차: 139.04837287793052
Moviepy - Building video ./videos/cartpole.mp4.
Moviepy - Writing video ./videos/cartpole.mp4



Moviepy - Done !
Moviepy - video ready ./videos/cartpole.mp4


## 7. 더 탐구해보기

- 어떤 모델과 환경이 호환되지 않는 이유가 무엇일까요?
- 학습된 모델의 성능을 높이기 위해 어떤 방법을 사용할 수 있을까요?
- 여러 환경과 모델을 사용하여 학습을 진행하고, 성능을 비교해보세요.
- 다양한 Wrappers, Callbacks를 활용하여 원하는 동작을 구현해보세요.